In [1]:
import requests
import pandas as pd
import datetime
import numpy as np
from progressbar import progressbar

In [2]:
URL_BASE = "https://vapi.binance.com"

In [3]:
def expiryDate(year, month, day, hour=9):
        return int(datetime.datetime(year=year, month=month, day=day, hour=hour).timestamp() * 1000)
    
def toExpiryDate(ts):
        dt = datetime.datetime.fromtimestamp(ts / 1000)
        return dt.strftime("%y%m%d")

# Demo Usage

In [10]:
response = requests.get(URL_BASE + "/vapi/v1/optionInfo")
data = response.json()
option_info = pd.DataFrame(data["data"])

In [11]:
response = requests.get(URL_BASE + "/vapi/v1/index", params={"underlying": "BTCUSDT"})
print(response.status_code)
index_price = response.json()["data"]
print(index_price)

200
{'indexPrice': '52661.42604'}


In [12]:
response = requests.get(URL_BASE + "/vapi/v1/ticker", params={"symbol": f"BTC-{EXPIRE_DATE}-60000-C"})
print(response.status_code)
ticker = response.json()["data"][0]
print(ticker)

200
{'symbol': 'BTC-210326-60000-C', 'priceChange': '37.35', 'priceChangePercent': '0.5747', 'lastPrice': '102.35', 'lastQty': '0.01', 'open': '65', 'high': '200', 'low': '65', 'volume': '1.6217', 'amount': '144.2124', 'openTime': 0, 'closeTime': 1616614710318, 'firstTradeId': 0, 'tradeCount': 7, 'strikePrice': '60000', 'expectedExercisePrice': '0'}


In [13]:
response = requests.get(URL_BASE + "/vapi/v1/mark", params={"symbol": f"BTC-{EXPIRE_DATE}-60000-C"})
print(response.status_code)
mark = response.json()["data"]
print(mark)

200
[{'symbol': 'BTC-210326-60000-C', 'markPrice': '69.01', 'bidIV': '1.0495', 'askIV': '1.3718', 'delta': '0.04402', 'theta': '-131.85087', 'gamma': '0.00003', 'vega': '3.02558', 'volatility': '1.2106', 'highPriceLimit': '2083.31', 'lowPriceLimit': '0.01'}]


In [14]:
response = requests.get(URL_BASE + "/vapi/v1/depth", params={"symbol": f"BTC-{EXPIRE_DATE}-60000-C"})
print(response.status_code)
depth = response.json()["data"]
depth

200


{'bids': [['30', '0.01'],
  ['22.63', '0.01'],
  ['10', '0.2'],
  ['2', '0.1'],
  ['1.07', '0.9246']],
 'asks': [['127.56', '0.02'],
  ['127.57', '1.1005'],
  ['162', '1.2151'],
  ['235', '0.001'],
  ['325', '0.4929'],
  ['495', '0.01'],
  ['2000', '0.01'],
  ['3000', '1.01'],
  ['4000', '0.01'],
  ['4600', '0.002'],
  ['4888', '0.025'],
  ['5000', '0.103'],
  ['15000', '0.025']]}

# Get Symbol Data

In [4]:
def get_options_info():
    response = requests.get(URL_BASE + "/vapi/v1/optionInfo")
    data = response.json()
    return pd.DataFrame(data["data"])

def get_index_price(underlying="BTCUSDT"):
    response = requests.get(URL_BASE + "/vapi/v1/index", params={"underlying": underlying})
    assert response.status_code == 200
    index_price = response.json()["data"]
    return index_price["indexPrice"]
    
def get_mark(symbol):
    response = requests.get(URL_BASE + "/vapi/v1/mark", params={"symbol": symbol}, allow_redirects=False)
    assert response.status_code == 200
    return response.json()["data"][0]

def get_ticker(symbol):
    response = requests.get(URL_BASE + "/vapi/v1/ticker", params={"symbol": symbol}, allow_redirects=False)
    assert response.status_code == 200
    return response.json()["data"][0]

def get_orderbook(symbol):
    response = requests.get(URL_BASE + "/vapi/v1/depth", params={"symbol": symbol, "limit": 1000})
    assert response.status_code == 200
    data = response.json().get("data", {})
    return {
        "asks": data.get("asks", []),
        "bids": data.get("bids", [])
    }

In [5]:
def repeat(n=10):
    def decorator(f):
        def wrapper(*args, **kwargs):
            for i in range(n): 
                try:
                    return f(*args, **kwargs)
                except AssertionError:
                    pass
            raise Exception("Repeat Failed.")
        return wrapper
    return decorator 


def decode_symbol(symbol):
    asset, expiry, strike, t = symbol.split("-")
    return pd.Series({
        "asset": asset,
        "expiry": int(expiry),
        "optionType": t,
        "strikePrice": int(strike)
    })

def symbol_orderbook(symbol, repeats=10):
    symbol_data = {
        "symbol": symbol,
        **decode_symbol(symbol)
    }
    order_book = repeat(n=10)(get_orderbook)(symbol)
    
    for price, qnty in order_book["asks"]:
        yield {
            "price": price,
            "qnty": qnty,
            "side": "ASK",
            **symbol_data
        }
        
    for price, qnty in order_book["bids"]:
        yield {
            "price": price,
            "qnty": qnty,
            "side": "BID",
            **symbol_data
        }
        
def symbols_orderbook(symbols, repeats=10):
    for s in symbols:
        yield from symbol_orderbook(s)

In [9]:
options_info = get_options_info()
for date in options_info.expiryDate.unique():
    print(toExpiryDate(date), datetime.datetime.fromtimestamp(date/1000).strftime("%H"))

210430 10
210409 10


In [10]:
expiry_dates = [
    #expiryDate(year=2021, month=3, day=26, hour=9),
    #expiryDate(year=2021, month=4, day=2, hour=10),
    expiryDate(year=2021, month=4, day=9, hour=10),
    expiryDate(year=2021, month=4, day=30, hour=10)
]

In [11]:
for expiry_date in expiry_dates:
    symbols = options_info[
        options_info.expiryDate == expiry_date
    ]["symbol"]


    obs = []
    for datarow in progressbar(symbols_orderbook(symbols)):
        obs += [datarow] 

    symbol_df = pd.DataFrame(obs)
    symbol_df["indexPrice"] = get_index_price()

    symbol_df.to_csv(f"orderbook_snapshot_{toExpiryDate(expiry_date)}.csv")

| | #                                               | 155 Elapsed Time: 0:00:19
| |                                             #   | 238 Elapsed Time: 0:00:24
